In [1]:
#importazione dei dati ------------------------------------------------------------------------------------------------------------

from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.linalg import toeplitz
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate

df = pd.read_csv (r'/Users/patrickpinta/Desktop/1datasetpanda/soybean.csv',header=0,error_bad_lines=False) 

#modellazione dei dati training set------------------------------------------------------------------------------------------------------------


xExpUsa= df.iloc[0:334,1:2]
xExpUsa = np.array(xExpUsa)
xExpBra=df.iloc[0:334,3:4]
xExpBra = np.array(xExpBra)
xExpArg=df.iloc[0:334,5:6]
xExpArg = np.array(xExpArg)
xImportChina=xExpUsa+xExpBra+xExpArg

xBimestri=df.iloc[0:334,17:22]
xBimestri = np.array(xBimestri)

xUsa=df.iloc[0:334,0:1]
xUsa = np.array(xUsa)
xBimInfExpUsa=xBimestri*xUsa

xBra=df.iloc[0:334,1:2]
xBra = np.array(xBra)
xBimInfExpBra=xBimestri*xBra

xArg=df.iloc[0:334,2:3]
xArg = np.array(xArg)
xBimInfExpArg=xBimestri*xArg


x=df.iloc[0:334,0:1]
x2=df.iloc[0:334,2:3]
x3=df.iloc[0:334,4:5]
x4=df.iloc[0:334,6:7]
x5=df.iloc[0:334,8:15]
x6=df.iloc[0:334,15:17]
x7=df.iloc[0:334,17:22]


xImportChina=xImportChina.tolist()
xImportChina = DataFrame (xImportChina,columns=['ImportChina'])
xBimInfExpUsa=xBimInfExpUsa.tolist()
xBimInfExpUsa = DataFrame (xBimInfExpUsa,columns=['bim1*expUsa','bim2*expUsa','bim3*expUsa','bim4*expUsa','bim5*expUsa'])
xBimInfExpBra=xBimInfExpBra.tolist()
xBimInfExpBra = DataFrame (xBimInfExpBra,columns=['bim1*expBra','bim2*expBra','bim3*expBra','bim4*expBra','bim5*expBra'])
xBimInfExpArg=xBimInfExpArg.tolist()
xBimInfExpArg = DataFrame (xBimInfExpArg,columns=['bim1*expArg','bim2*expArg','bim3*expArg','bim4*expArg','bim5*expArg'])
x=x.join(xImportChina)
x=x.join(x2)
x=x.join(x3)
x=x.join(x4)
x=x.join(x5)
x=x.join(x6)
x=x.join(xBimInfExpUsa)
x=x.join(xBimInfExpBra)
x=x.join(xBimInfExpArg)
x=x.join(x7)


y= df.iloc[0:334,23:24]

#stardadizzazione dei dati training set------------------------------------------------------------------------------------------------------------

scaler = MinMaxScaler(feature_range=(-1,1))

x_train_stand=(scaler.fit_transform(x))
    
#modello dati training set------------------------------------------------------------------------------------------------------------    
    
x_train_stand, y = np.array(x_train_stand), np.array(y)
x_train_stand = sm.add_constant(x_train_stand)

#procedura per ols model ------------------------------------------------------------------------------------------------------------

ols_model = sm.OLS(y, x_train_stand)
ols_results = ols_model.fit(cov_type='HC1') 
print(ols_results.summary())

reg = linear_model.LinearRegression()
reg= reg.fit(x_train_stand,y)

#forecast ------------------------------------------------------------------------------------------------------------

ypredTrain = reg.predict(x_train_stand)

print('r2 train: ',r2_score(y,ypredTrain))
print('mse train: ',mean_squared_error(y,ypredTrain))

scores=cross_validate(reg, x_train_stand, y, cv=5,scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)
print('r2 cv: ', scores['test_r2'].mean())
print('mse cv: ', abs(scores['test_neg_mean_squared_error'].mean()))











                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.748
Method:                 Least Squares   F-statistic:                     27.08
Date:                Fri, 14 May 2021   Prob (F-statistic):           4.34e-72
Time:                        16:02:57   Log-Likelihood:                -1979.9
No. Observations:                 334   AIC:                             4030.
Df Residuals:                     299   BIC:                             4163.
Df Model:                          34                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        673.0234    142.812      4.713      0.0

## 